In [2]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

# Read the excel file and 3 sheets
df=pd.read_excel("Dataset.xlsx",sheet_name=['Day-1','Day-2','Day-3','List'])


# Dictionary for activity code mapping
act_code={"Eating / Drinking":"ED145", "Education / Lectures":"EL642","Exercise":"X893", "Housework":"H179","Leisure":"L418",
           "Reading":"R523","Sleep":"S801","Travelling":"T695","Using Devices":"UD415", "Paid Work":"PW101","Coursework": "CW982",
          "Other":"O733"}

#Fetching the date of the diary entry made at the top of each sheet
df['List']['Date']=pd.to_datetime(df['List']['Date']).dt.date

# Dictionary for UniBathWeekNo mapping and dates
week={df['List'].iloc[i,6]:df['List'].iloc[i,7] for i in range(len(df['List']))}

#Returns the activity code for activity name input
def activity_code(row):
    if str(row) == 'nan':
        return np.nan
    else:
        row=row.strip()
        return act_code[row]
    
#Returns week no for date input   
def week_no(row):
    for k,v in week.items():
        end=k+timedelta(days=6)
        if (k<= row.date() <=end):
            return week[k]
        
#Returns day of the week for date input      
def day_week(row):
    row=row.strftime('%A')
    return row
    
#Returns duration for start and end time inputs
def duration(data):
    duration_m=[]
    for i in range(len(data)):
        
        d1 = datetime.datetime.combine(data['StartDate'][i],data['StartTime'][i])
        d2 = datetime.datetime.combine(data['StartDate'][i],data['EndTime'][i])
       
        d=d2-d1
        h=(str(d)[-8:-3])
        
        h=h.split(":")
        if len(h)==2:
            duration_m.append(int(h[1])+int(h[0])*60)
        else:
            duration_m.append(h[1])
    return(duration_m)
    
#Returns the rounded of value to the nearest 15 mins
def round_15(d):
    
    rem = d % 15
    if rem < 5:
        n = int(d / 15) * 15
    else:
        n = int((d + 15) / 15) * 15
    return n

#Returns dataframe in the required format given excel sheet name

def create_dataset(sheet):
    data=pd.DataFrame(columns=['PrimaryActivityCode','SecondaryActivityCode','DescriptiveLabel','StartDate','UniBathWeekNo','DayOfWeek'
                               ,'StartTime','EndTime','DurationMins','EnjoymentScore','AdditionalData1','AdditionalData2'])
    
    date=df[sheet].keys()[2]
    col=df[sheet].iloc[0,:]
    df[sheet]=df[sheet].set_axis(col,axis=1, inplace=False)
    df[sheet]=df[sheet].drop(index=[0,1])
    df[sheet]=df[sheet].reset_index(drop=True)

    

    # print(df['Day-1'].columns)
    data['PrimaryActivityCode']=(df[sheet]['Primary Activity'].apply(activity_code))
    data['SecondaryActivityCode']=df[sheet]['Secondary activity'].apply(activity_code)
    data['DescriptiveLabel']=df[sheet]['Activity Description']
    
    data['StartDate']=date
    data['StartDate'].iloc[0]=date-timedelta(1)
    data['StartDate']=pd.to_datetime(data['StartDate'])

    data['UniBathWeekNo']=data['StartDate'].apply(week_no)

    data['DayOfWeek']=data['StartDate'].apply(day_week)
    

    data['StartTime']=df[sheet]['Start Time']

    data['EndTime']=df[sheet]['End Time']
        

    data['DurationMins']=pd.Series(duration(data))

    data['DurationMins']=data['DurationMins'].apply(round_15)

    data['EnjoymentScore']=df[sheet]['Enjoyment Score']

    data['AdditionalData1']=df[sheet]['Did you wear a mask?']

    data['AdditionalData2']=df[sheet]['Were you with:']
    return data

#Create dataframe from sheet data
df1=create_dataset('Day-1')

df2=create_dataset('Day-2')

df3=create_dataset('Day-3')

# Concat the rows of 3 dataframes into a single dataframe
dataset=pd.concat([df1,df2,df3])

#displays the final dataset
display(dataset)


#Convert to csv
dataset.to_csv("D:\s2\ADS\completed_data.csv",index=False)


C:\Users\sruth\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\sruth\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,PrimaryActivityCode,SecondaryActivityCode,DescriptiveLabel,StartDate,UniBathWeekNo,DayOfWeek,StartTime,EndTime,DurationMins,EnjoymentScore,AdditionalData1,AdditionalData2
0,S801,NaN,woke up,2022-02-10,19,Thursday,23:00:00,07:00:00,480,-2,No mask,Alone
1,H179,NaN,Getting Ready,2022-02-11,19,Friday,07:00:00,07:30:00,30,0,No mask,Alone
2,ED145,NaN,Breakfast,2022-02-11,19,Friday,07:35:00,08:15:00,45,1,No mask,Alone
3,H179,NaN,Packing Essentials,2022-02-11,19,Friday,08:15:00,08:40:00,30,-1,No mask,Alone
4,L418,UD415,Browsing Internet,2022-02-11,19,Friday,08:45:00,09:15:00,30,2,No mask,Alone
...,...,...,...,...,...,...,...,...,...,...,...,...
21,CW982,UD415,Assignment,2022-02-23,21,Wednesday,19:10:00,20:00:00,60,1,No mask,Alone
22,ED145,NaN,Dinner,2022-02-23,21,Wednesday,20:00:00,20:30:00,30,1,No mask,Alone
23,H179,NaN,Cleaning,2022-02-23,21,Wednesday,20:30:00,20:45:00,15,-2,No mask,Alone
24,CW982,UD415,Assignment,2022-02-23,21,Wednesday,20:50:00,21:50:00,60,1,No mask,Alone


PermissionError: [Errno 13] Permission denied: 'D:\\s2\\ADS\\dataset_final.xlsx'